In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'titanic']


In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [4]:
train_dir = "../data/10-monkey-species/training/training"
valid_dir = "../data/10-monkey-species/validation/validation"
label_file = "../data/10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

True
True
True
['n0', 'n7', '.DS_Store', 'n9', 'n8', 'n6', 'n1', 'n4', 'n3', 'n2', 'n5']
['n0', 'n7', 'n9', 'n8', 'n6', 'n1', 'n4', 'n3', 'n2', 'n5']


In [5]:
labels = pd.read_csv(label_file,header=0)
labels

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata\t,mantled_howler,131,26
1,n1,erythrocebus_patas\t,patas_monkey,139,28
2,n2,cacajao_calvus\t,bald_uakari,137,27
3,n3,macaca_fuscata\t,japanese_macaque,152,30
4,n4,cebuella_pygmea\t,pygmy_marmoset,131,26
5,n5,cebus_capucinus\t,white_headed_capuchin,141,28
6,n6,mico_argentatus\t,silvery_marmoset,132,26
7,n7,saimiri_sciureus\t,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps\t,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26


In [6]:
height = 128
width = 128
channels = 3
batch_size = 64
num_classes = 10

In [9]:
train_datagen =  keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./ 255.
    , rotation_range = 40
    , width_shift_range = 0.2
    , height_shift_range = 0.2
    , shear_range = 0.2
    , zoom_range = 0.2
    , horizontal_flip=True
    ,fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir
    , target_size = (height,width)
    , batch_size = batch_size
    , shuffle = True
    , class_mode = 'categorical'
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255.)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir
    , target_size = (height,width)
    , batch_size = batch_size
    , shuffle = True
    , class_mode = 'categorical'
)

train_num = train_generator.samples
valid_num = valid_generator.samples

print(train_num)
print(valid_num)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098
272


In [11]:
for i in range(2):
    x , y = train_generator.next()
    print(x.shape,y.shape)
    

(64, 128, 128, 3) (64, 10)
(64, 128, 128, 3) (64, 10)


In [15]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[width,height,channels]) # 卷积层
    , keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu') # 卷积 提取32个特征
    , keras.layers.MaxPool2D(pool_size = 2) # 池化 降低图片大小
    
    , keras.layers.Conv2D(filters = 64 , kernel_size = 3 , padding = 'same') # 卷积 提取64个特征
    , keras.layers.Conv2D(filters = 64 , kernel_size = 3 , padding = 'same')
    , keras.layers.MaxPool2D(pool_size = 2) # 池化 
    
    , keras.layers.Conv2D(filters = 128 , kernel_size = 3 , padding = 'same') # 卷积 提取128个特征
    , keras.layers.Conv2D(filters = 128 , kernel_size = 3, padding = 'same')
    , keras.layers.MaxPool2D(pool_size = 2) #  池化 
    
    , keras.layers.Flatten() # 展开
    , keras.layers.Dense(128,activation='relu') # 全连接
    , keras.layers.Dense(num_classes,activation='softmax') # softmax输出结果 
])

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 128)       7

```
epochs = 10
history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num // batch_size)
```